# Chapter 8 - Stereovision, Trinagulation, Feature Correspondance, Disparity Map

In this acticle we introduce the topic of stereo vision which is the application of mutiple camera views to get information about the depth of the view. Using stereo Visione one can derive the world locaton of a point from its images in different camera views. 


## Multiple View Geometry

There are different variations of multiple view geometry as this term encompasses all aplication working with multiple view of the save object from different angles. This can be from stereo cameras which have multiple carmeras in the same devices or different views from different devices. So let un defined two different terms which are important to keep in mind for the rest of the article.

**3D reconstruction from multiple views**: For this application we assume that the camera intrinsics K are known. Also We assume that we knoe the Ratation and the Transaltion between different views. This can be by using some IMUs or other mechanisms. The Goal is to reconstruct the 3D structure of the objecft drim many different images.

**Structure From Motion (SFM)**: In this application we do now know any intrinsics K or any Translation or Rotation. Therefore the goal is to recover the intrinsics as well as the camera pose together ith the 3D structure of the scene.

In the case of exact 2 views we call the first case **Depth from stereo** and the second one **2-view Structure from Motion**.

### Depth from Stereo

For a single camera only back-project the ray on which the image point lies but we can not determine at which depth the actual object is located. An illustraiotn for that you can see in the left side in Figure 1. When using two cameras as we do in stereo cameras we have such a ray-back-projection originating from different locations in the world. As a result we can just see where the rays from the left camera intersect with the ones from the right camera to know at which depth the object lies.

![Perspective and Stereo](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_8/1_projection_and_stereo.png)
*Figure 1: Perspective and Stereo [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/07_multiple_view_geometry_1.pdf)*

Our brain also work like this and thats the reasons why most amymals and humans have two eyes. This is so that they are able to estimate the depth of the secene they are seeing. Our brain mixes the two images that we are seeing, one from each eye, into one single image. This is similar to the processing our brain makes to turn the observed images as the eyes observe the world in an upside-dwn view.

You can experience the effect of stereo vision simpli by holting up one finger infornt of your eyes and then closing the left eye only, and short after that the right weye only. You will observe that the fingers seems to jumping from left to right.
This observed horizontal displacement is called **disparity**.



### Disparity

The displacement between two images allows us to percept depth. The displacement in the image is always along the same axis as the displacement of the perceptors. Usually tthe perceptors are displaced horizonally as for the example with the eyes of animals of also most standard stereo cameras. One application where this is used are the anaglyphs. These are the images with red and blue thint which appear with depth when you look trough soe glasses where infron of the eye there is a blue filter and infront of the othere there is a red filter.

Another application is the stereogram where you can see certain images in depth when focusing behind the actual image. In the example below you have to focus about 10cm behind the image plane to then hopefully see the plant. For this effect the stereo vision is needed. You can try it out by clonsing one eye when you have been able to focus correctly to see tha plant. Once you close one eye you will instatly lose the vision of the plant.

![Stereogram](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_8/2_stereogram.png)
*Figure 2: Stereogram [source](http://Image from magiceye.com)*


### Stereo Vision

The basic principle of stereo vision is to recosntruct the 3D position of a point by intersecting its rays thouth its images in different views. For this we need the position of the camera so that we know the abslute worl position of the ray. also we need to knwo which points of the different view images to correspond (point correspondence)
In general the different the cameras image planes can be orientated independently from each other. However the problem becomes easier when the imageplanes are aligned and also when the same camera intrinsics can be applied to all images. Meaning the cameras are identical.

![Stereo simple and general case](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_8/3_stereo_cases.png)
*Figure 3: Stereo simple and general case [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/07_multiple_view_geometry_1.pdf)*

In figure 4 we want to find an expression forthe distance from the image to the object which is denoted as $Z_P$. We can see that both cameras $C_l$ and $C_r$ have the same focal length $f$. Also they are prefectly allinged with a horizontal displacement $b$, which we call **baseline**. From the similarity of the triangles in green and yellow we know that the relation betwen the length of the red lines must be the same as the relation of the lengths of the blue lines. Therefore we know:

\begin{align*}
\dfrac{f}{Z_P} &= \dfrac{u_l}{X_p} \\
\dfrac{f}{Z_P} &= \dfrac{-u_r}{b-X_p}
\end{align*}

From this we can derive:

\begin{align*}
Z_P = \dfrac{bf}{u_l-u_r} \\
\end{align*}

$u_l-u_r$ is what we have previously introduced as **disparity**, the displacement of the same object on the different images in pixel coordinates.

![Stereo Vision simple](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_8/4_stereo_vision_simple.png)
*Figure 4: Stereo Vision simple [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/07_multiple_view_geometry_1.pdf)*

For practical applications some questions may arise. For example what is optimal baseline distance? Because if the it is too large, then the minimum measurable depth increases and close object can not be searched for. If the baseline is too small then there is a large depth error.

![Large and Small Baseline](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_8/5_large_and_small_baseline.png)
*Figure 5: Large and Small Baseline [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/07_multiple_view_geometry_1.pdf)*



In reality the case pictured in Figure 4 barely happes where both camera planes are prefectly aligned. In Practice two identical cameras do not exist and aligning then perfectly is very hard. Also because external infulences like pressure and temparature can deform the materials. Therefore to use stereo camera we need the extrinsic parameters like rotation and translation as well as the intrinsic parameters of each camera which are the focal length, optical center and radial disortion. These intrinsics we can get through the calibration methods.

### Trinagulation

Triangulation describes the process of finding the 3D position of a point given known camera location and rotation and a set of corresponding image location of that point. The goal is to instersect multiple viual rays corresponding to the image and camera center locations. The point where these rays intersect is then the 3D location of the point.

#### Linear Approximation

![Linear Approximation](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_8/6_linear_approximation.png)
*Figure 6: Linear Approximation [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/07_multiple_view_geometry_1.pdf)*

Recalling some knowledge about cross (vector) product and dot product:
- the dot product of twh vectors which are perpendicular to each others is 0
- the cross product of two vectors that are parallel is 0

The cross product can also be expressed as a **skew-symmetric matrix**.

![Skew Symmetric Matrix](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_8/7_skew_symmetric_matrix.png)
*Figure 7: Skew-Symmetric Matrix [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/07_multiple_view_geometry_1.pdf)*

We can express the image location in terms of world-coordinates as a dot product. We express the 3D position of the point in terms of the world coordinates with origin in the camera position of the left camera. Thefore cross product of $p_i$ and $M_i$ describes the normal vector onto the epipolar plane. As a result the position vector for the 3D position of the point is perpendicular to that normal vector of the epipolar plane. As a result we can set the ebelow equations to 0.

![triangulation equation part1](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_8/8_triangulation_equation_1.png)
![triangulation equation part2](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_8/8_triangulation_equation_2.png)
*Figure 8: Triangulation [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/07_multiple_view_geometry_1.pdf)*

In the general case it might happen that the rays of images do not intersect. In such a case the 3D position of the point is determined as the mid point of the shortest segment connecting the two rays.

![Shortest Segment Midpoint](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_8/9_shortest_segment.png)
*Figure 9: Shortest Segment Midpoint [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/07_multiple_view_geometry_1.pdf)*

#### Non-Linear Approximation

A more complex but more accurate description of the 3D position of the point is using the non-linear approximation.
Here we want to find the point $P$ such that is minimizes the **Sum of Squared Reprojection Errors (SSRE)**.

\begin{align*}
SSRE = {\lVert p_1- \pi_1(P) \rVert }^2 + {\lVert p_2- \pi_2(P) \rVert}^2
\end{align*}

where $\pi_i(P)$ is the projection of P onto the i-th camera plane and ${\lVert p_i- \pi_i(P) \rVert }^2$ is the **Reprojection Error**.

![Reprojection Error](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_8/10_reprojection_error.png)
*Figure 10: Reprojection Error [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/07_multiple_view_geometry_1.pdf)*


### Correspondence Problem

Given a point of the image of the left camera $p_l$, how can we find that point easily and fast on the right image ($p_r$)?
One approach would be the take a patch around $p_l$ and scan the right iamge and compare all patches using similarity measure again the patch of $p_l$. We can use similarity measures like (Z)NCC, (Z)SSD, (Z)SAD or Census Transform. However scanning the whole image is very slow since we try to compare an extensive amount of patches. However we can constrain the set of possible patches to speed up the process. The solution to this is called **epipolar line**. This line describes the projection of the rays of the camera in image one and the point onto the second camera image. On other words: From the image location of a point and the camera location on the single camera case we know that the point in 3D has to lie on the ray from the camera center through the image point. However we do not know at with distance (depth). We can now project this ray onto the camera plane of the second camera with results in a line, the epipolar line. We can also draw a line connecting both camera center location. The points where this line intersects the image plane is called **epipoles**. Since in the stereo camera case we have two cameras and therefore one line and two image planes, we have two epipoles.

![Epipolar Line](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_8/11_epipolar_line.png)
*Figure 11: Epipolar Line [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/07_multiple_view_geometry_1.pdf)*

The plane that is span by the the two camera point and the 3D point is called epipolar plane. We can also define the epipolar line as the intersection of the epipolar plan with the camera plane. We call the constraint that the position of corresponding points have to lie in their epipolar line the **epipolar constraint**. Using this contraint we can reduce the region to search for a corresonding point to a 1D search on the epipolar line.

![Epipolar Plane](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_8/12_epipolar_plane.png)
*Figure 12: Epipolar Plane [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/07_multiple_view_geometry_1.pdf)*

![Examples](https://github.com/joelbarmettlerUZH/PyVisualOdometry/raw/master/img/chapter_8/13_epipolar_examples.png)
*Figure 12: Examples [source](http://rpg.ifi.uzh.ch/docs/teaching/2019/07_multiple_view_geometry_1.pdf)*



### Stereo Rectifications

### Exercise 5

Work description: Stereo vision: rectification, epipolar matching,
disparity, triangulation

TODO!!!